In [ ]:
import xarray as xr

In [ ]:
ds = xr.open_dataset('fram_data/roho160_his_0001.nc')

In [ ]:
# da_stacked containes values of zeta < -1 and their coordinates
da = ds.zeta.isel(ocean_time=1)
da_stacked = da.where(da<-1).stack(x=['eta_rho','xi_rho'])
da_stacked = da_stacked[da_stacked.notnull()]

In [ ]:
da.where(da<-1).plot(figsize=(20, 10))

In [ ]:
da.where(da<-1).isel(eta_rho=slice(650, 750), xi_rho=slice(1400, -1)).plot(figsize=(20, 10))